In [1]:
import matplotlib.pyplot as plt  # matplotlib version: 3.6.3
import numpy as np
import pandas as pd
import scipy.fft as fft
import plotly.graph_objects as go
import plotly.express as px
import webbrowser
from scipy.optimize import curve_fit
import os
import re
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import sympy as sp

In [23]:
opt_parameters = [   34.20949562,  1317.99024017,  -156.27590486,  -544.54377573,
       -5732.4692288 ,   282.56780882,  9396.56391293,  -148.9753875 ,
         -51.41313683,  1244.93126392]

In [24]:
def aggIn(KP, KD, args):
    if len(args) < 10:
        raise ValueError("Wrong number of arguments")
    
    return args[0] + args[1]*KP + args[2]*KD + args[3]*KP*KD + args[4]*KP**2 + args[5]*KD**2 + args[6]*KP**3 + args[7]*KD**3 + args[8]*KP*KD**2 + args[9]*KP**2*KD

In [27]:
z = 110
KP = 0.15
KD = sp.Symbol('KD')

eq = sp.Eq(aggIn(KP, KD, opt_parameters), z)
sol = sp.solve(eq, KD)

print([s for s in sol if s.is_real and s >= 0.001 and s <= 1])


[0.141603536646224, 0.851685640194285 - 0.665355539897431*I, 0.851685640194285 + 0.665355539897431*I]
